# **Data Cleaning:**
Summarize what we did in Excel, and some stuff moving into python

In [2]:
# further data cleaning in pandas
import pandas as pd

table6 = pd.read_csv("Matthew's Data/Table6.csv")
table8 = pd.read_csv("Matthew's Data/Table8.csv")
table9 = pd.read_csv("Matthew's Data/table9.csv")

# create list of all strings that contain the names of communities in table 8
community_names = table8["Community"].unique().tolist()

# filter the csv based on if it exists in the observed commmunities
table9_filtered = table9[table9["Area Name"].str.contains('|'.join(community_names), regex=True)]
table9_filtered

# clean again 
#print(len(community_names))
#print(len(table9_filtered.index))s
#for i in community_names:
    #if not table9_filtered["Area Name"].str.contains(i).any():
        #print(i, "is not in here")

,Area Name,2021,2022
4,Bow Island,2106,2129
10,Raymond1 *,4334,4455
18,Coaldale3 *,9073,9179
19,Taber,9177,9325
26,Bassano,1250,1255
...,...,...,...
424,Spirit River No. 133,666,669
426,Spirit River,1022,998
428,Fairview No. 136,1626,1603
429,Fairview,2912,2914


In [ ]:
# TODO clean population estimates so it only includes the communities that we want
# TODO fix tables 2b-2 and 2b
# TODO start working on faceted graph
# TODO for population before graph, make as ranges

Ok so here is the shit that I am going to do for my part, please listen to the checklist ANALYSIS
- make **Faceted Graph** for table 8 (and 6 now)
    - write observations if any
    - **MAINLY, WHAT RELATION BETWEEN RENT AND VACANCY IS THERE??**
        - **HOW DOES THIS CHANGE BETWEEN COMMUNITY SIZES??**

- <u>Sampled Towns</u> --> how might these be useful?
    - Banff, Drumheller, Slave Lake, 
        - *State why these are good representatives in the sample for the whole communities* in the notebook
            - sdf
    - Analyze overall Rental Range from table 2b and vacancy rates?


New changes
- removed commas in thousands from Table 8 --> can read as numerical values now
- removed "Overall Vacancy Rates" from Table 8, since this is covered in table 6 and is more accurate there, just need the weighted rents between years in table 8\
- added "population" which we want to analyze how town population sizes might also be related to costs and shit
    - add to faceted graph as an outer variable

-------

# **Visualizations + Analysis:**

In [ ]:
# faceted chart